In [289]:
# import packages 
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import os 
import sys
import seaborn as sns
import scipy as sp
import scipy.stats as stats
from scipy.stats import norm
%matplotlib inline

# import pre-processing modules 
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel 

# import classifiers
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC

# import evaluation modules
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report

In [290]:
test_set = pd.read_csv(r"C:/Users/Yehonatan/PycharmProject/DS/projects/titanic/test.csv")
train_set = pd.read_csv(r"C:/Users/Yehonatan/PycharmProject/DS/projects/titanic/train.csv")
df_original = pd.concat([train_set,test_set], axis=0, ignore_index=True )


In [291]:
df_train = (train_set.copy()).rename(columns=str.lower)
df_test = (test_set.copy()).rename(columns=str.lower)
df = (df_original.copy()).rename(columns=str.lower)


#df.info()
#df_test.info()

In [292]:
# parse title and fam name from df 
def parse_name(df):
    last_name = df['name'].apply(lambda x : x.split(', ')[0])
    temp = df['name'].apply(lambda x : x.split(', ')[1])
    title = temp.apply(lambda x : x.split('.')[0])
    return title, last_name



In [293]:
# work on df nan and then concat with original df_train and sort by ID
age_nan = df[df['age'].isna()]
#age_nan.info()
#age_nan = age_nan.sort_values(by=['sex'],ascending=True)
#age_nan = age_nan.sort_values(by=['sibsp'],ascending=False).reset_index(drop=True)
print('df updated nans',df['age'].isna().sum())
print('len age_nan',len(age_nan))


df updated nans 263
len age_nan 263


In [294]:
title, last_name = parse_name(age_nan)
title = title[title == 'Master']
master_index = title.index.to_numpy()

df_young = df[df["age"] < 15 ]
child_mean = round(df_young['age'].mean(),2)
       
for i in master_index:
    df.at[i,'age'] = child_mean 

print('df updated nans',df['age'].isna().sum())
print('len age_nan before update',len(age_nan))

age_nan = df[df['age'].isna()] # update the age_nan

print('len age_nan',len(age_nan))

df updated nans 255
len age_nan before update 263
len age_nan 255


In [295]:
l_p_t = age_nan[age_nan['sibsp'] == 0]
lone_pass = l_p_t[l_p_t['parch'] == 0] # passengers who travel alone and cant determine their age by relatives
#lone_pass

# calc mean by gender
df_male = df[df['sex'] == 'male']
df_female = df[df['sex'] == 'female']
male_mean = round(df_male['age'].mean(),2)
female_mean = round(df_female['age'].mean(),2)

lone_pass_male = lone_pass[lone_pass['sex'] == 'male']
lone_pass_female = lone_pass[lone_pass['sex'] == 'female']

l_p_m_index = (lone_pass_male.index).to_numpy()
l_p_f_index = (lone_pass_female.index).to_numpy()

for m in l_p_m_index:
    df.at[m,'age'] = male_mean

for f in l_p_f_index:
    df.at[f,'age'] = female_mean
    
age_nan = df[df['age'].isna()] # update the age_nan

print('len age_nan',len(age_nan))


len age_nan 56


In [191]:
#age_nan = age_nan.sort_values(by=['sibsp'],ascending=False)

#age_nan

In [343]:
part_fam = age_nan[age_nan['parch'] == 0]
title, last_name = parse_name(part_fam)
title_index = (title.index).to_numpy()

mrs_nan_lastname = [] # find the fa,ilr names of the mrs with nans
index_mrs_nan = []
for t in range(len(title)):
    if (list(title)[t]) == 'Mrs':
        mrs_nan_lastname.append(list(last_name)[t])
        index_mrs_nan.append(title_index[t])


husband_index =[] 
h_o_t = df[df['sex'] == 'male']
h_o_tt = h_o_t[h_o_t['sibsp'] == 1] # h_o = husband only df
h_o = h_o_tt[h_o_tt['parch'] == 0]

title_h_o, fam_nam_h_o = parse_name(h_o)
h_o_index = (title_h_o.index).to_numpy()

print(mrs_nan_lastname)
#print(list(fam_nam_h_o))

for l in range(len(mrs_nan_lastname)):
    for i in range(len(fam_nam_h_o)):
        if list(mrs_nan_lastname)[l] == list(fam_nam_h_o)[i]:
            husband_index.append(h_o_index[i])
            
for i in husband_index:           
    print(df.loc[i,:])
    
# I need to add the age of the husbands to wifes


['Spencer', "O'Brien", 'Frauenthal', 'Davison', 'Meyer', 'Thorneycroft', 'Kenyon', 'Caram', 'Taylor', 'Goldenberg', 'Khalil']
passengerid                             1208
survived                                 NaN
pclass                                     1
name           Spencer, Mr. William Augustus
sex                                     male
age                                     57.0
sibsp                                      1
parch                                      0
ticket                              PC 17569
fare                                146.5208
cabin                                    B78
embarked                                   C
Name: 1207, dtype: object
passengerid                    365
survived                       0.0
pclass                           3
name           O'Brien, Mr. Thomas
sex                           male
age                            NaN
sibsp                            1
parch                            0
ticket                      

In [ ]:
for l in range(len(mrs_nan_lastname)):
    #print('1',mrs_nan_lastname[l])
    for i in range(len(h_o_title)):
        #print('2',h_o_last_name[i])
        if mrs_nan_lastname[l] == h_o_last_name[i]:
            #print('1',mrs_nan_lastname[l])
            #print('2',h_o_last_name[i])
            husband_index.append(h_o_index[i])

print(mrs_nan_lastname)
for i in range(len(husband_index)):           
    print(df.loc[i,:])

In [ ]:
h_o_title, h_o_last_name = parse_name(h_o) 
h_o_index = (h_o_title.index).to_numpy()


for l in range(len(mrs_nan_lastname)):
    #print('#',mrs_nan_lastname[l])
    for i in range(18):
        if mrs_nan_lastname[l]  == fam_name_1[i]:
            #print(mrs_nan_lastname[l])
            #print(fam_name_1[i])
            #print(df.loc[i,:])    
            continue

In [ ]:
h_o_title, h_o_last_name = parse_name(h_o) 
h_o_index = (h_o_title.index).to_numpy()

for l in range(len(mrs_nan_lastname)):
    #print('1',mrs_nan_lastname[l])
    for i in range(len(h_o_title)):
        #print('2',h_o_last_name[i])
        if mrs_nan_lastname[l] == h_o_last_name[i]:
            husband_index.append(h_o_index[i])

#print(mrs_nan_lastname)
#for i in range(len(husband_index)):           
    #print(df.loc[i,:])